In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [5]:
def validar_variable(variable_input, df):
    try:
        variable_input = int(variable_input)
        if variable_input in df['idVariable'].values:
            return variable_input
        else:
            return None
    except ValueError:
        return None

def validar_fecha(fecha_str):
    try:
        return datetime.strptime(fecha_str, "%Y-%m-%d")
    except ValueError:
        return None

def dividir_rango_fechas(desde, hasta):
    """Divide un rango de fechas en segmentos de hasta un año."""
    inicio = []
    final = []
    fecha_actual = desde
    while fecha_actual < hasta:
        siguiente = min(fecha_actual + timedelta(days=365), hasta)
        inicio.append(fecha_actual)
        final.append(siguiente)
        fecha_actual = siguiente + timedelta(days=1)
    return inicio, final



In [3]:
principales_variables = pd.read_csv('principalesVariables.csv')
# URL de la API
print (principales_variables[['idVariable']+['descripcion']])


    idVariable                                        descripcion
0            1  Reservas Internacionales del BCRA (en millones...
1            4  Tipo de Cambio Minorista ($ por USD) Comunicac...
2            5  Tipo de Cambio Mayorista ($ por USD) Comunicac...
3            6             Tasa de Política Monetaria (en % n.a.)
4            7     BADLAR en pesos de bancos privados (en % n.a.)
5            8       TM20 en pesos de bancos privados (en % n.a.)
6            9  Tasas de interés de las operaciones de pase ac...
7           10  Tasas de interés de las operaciones de pase pa...
8           11  Tasas de interés por préstamos entre entidades...
9           12  Tasas de interés por depósitos a 30 días de pl...
10          13  Tasa de interés de préstamos por adelantos en ...
11          14            Tasa de interés de préstamos personales
12          15      Base monetaria - Total (en millones de pesos)
13          16       Circulación monetaria (en millones de pesos)
14        

In [ ]:

while True:
    while True:
        variable = input("Ingresa una variable (idVariable): ")
        variable = validar_variable(variable, principales_variables)
        if variable is not None:
            print(f"Variable válida: {variable}")
            break
        else:
            print("La variable ingresada no es válida. Inténtalo de nuevo.")

    while True:
        desde_str = input("Ingresa la fecha inicial (formato yyyy-mm-dd): ")
        desde = validar_fecha(desde_str)
        if desde:
            print(f"Fecha inicial válida: {desde_str}")
            break
        else:
            print("Formato de fecha inválido. Inténtalo de nuevo.")

    while True:
        hasta_str = input("Ingresa la fecha final (formato yyyy-mm-dd): ")
        hasta = validar_fecha(hasta_str)
        if hasta and hasta > desde:
            print(f"Fecha final válida: {hasta_str}")
            break
        else:
            print("Formato de fecha inválido o la fecha es anterior a la inicial. Inténtalo de nuevo.")

    # Dividir el rango si es necesario
    inicio, final = dividir_rango_fechas(desde, hasta)
    print(f"Rangos generados: {list(zip(inicio, final))}")

    

    for i, f in zip(inicio, final):
        url = f"https://api.bcra.gob.ar/estadisticas/v2.0/datosvariable/{variable}/{i.strftime('%Y-%m-%d')}/{f.strftime('%Y-%m-%d')}"
        try:
            response = requests.get(url, verify=False)

            # Convertir la respuesta JSON a un objeto de Python
            data = response.json()
        
            # Extraer la lista 'results'
            status = data.get("status")
            if (status == 200):
                print('La URL es valida.')
            else:
                print(data.get('errorMessages'))
                break

        except KeyError as e:
            print(f"Error procesando los datos para el rango {i} - {f}: {e}")

    if (status == 200):
        print('Todos los datos son validos')
        break
    else:
        print(data.get('errorMessages'))



Variable válida: 1
Fecha inicial válida: 2020-01-01
Fecha final válida: 2023-01-01
Rangos generados: [(datetime.datetime(2020, 1, 1, 0, 0), datetime.datetime(2020, 12, 31, 0, 0)), (datetime.datetime(2021, 1, 1, 0, 0), datetime.datetime(2022, 1, 1, 0, 0)), (datetime.datetime(2022, 1, 2, 0, 0), datetime.datetime(2023, 1, 1, 0, 0))]
La URL es valida.
La URL es valida.
La URL es valida.
200
Todos los datos son validos


In [8]:

# Crear un DataFrame concatenado
dataframes = []

for i, f in zip(inicio, final):
    url = f"https://api.bcra.gob.ar/estadisticas/v2.0/datosvariable/{variable}/{i.strftime('%Y-%m-%d')}/{f.strftime('%Y-%m-%d')}"
    try:
        response = requests.get(url, verify=False)
        response.raise_for_status()  # Verifica si hubo errores en la solicitud

        # Convertir la respuesta JSON a un objeto de Python
        data = response.json()
        
        # Extraer la lista 'results'
        results = data.get("results", [])
        
        # Convertir la lista de diccionarios en un DataFrame
        if results:
            df = pd.DataFrame(results)
            dataframes.append(df)
            print(f"Datos obtenidos para el rango {i} - {f}: {len(results)} registros.")
        else:
            print(f"Sin datos para el rango {i} - {f}.")

    except requests.exceptions.RequestException as e:
        print(f"Error al conectar con la API para el rango {i} - {f}: {e}")
    except KeyError as e:
        print(f"Error procesando los datos para el rango {i} - {f}: {e}")

# Concatenar los DataFrames obtenidos
if dataframes:
    VariableElegida = pd.concat(dataframes, ignore_index=True)
    print("DataFrame concatenado:")
    print(VariableElegida.head())
else:
    print("No se obtuvieron datos de la API.")

Datos obtenidos para el rango 2020-01-01 00:00:00 - 2020-12-31 00:00:00: 240 registros.
Datos obtenidos para el rango 2021-01-01 00:00:00 - 2022-01-01 00:00:00: 244 registros.
Datos obtenidos para el rango 2022-01-02 00:00:00 - 2023-01-01 00:00:00: 244 registros.
DataFrame concatenado:
   idVariable       fecha    valor
0           1  2020-01-02  44839.0
1           1  2020-01-03  44872.0
2           1  2020-01-06  44904.0
3           1  2020-01-07  45080.0
4           1  2020-01-08  45162.0
